In [2]:
import json

In [3]:
with open('heroes.json', 'r') as fp:
    heroesData = json.load(fp)

In [11]:
recommendations = [23,45,67,58,54]
recommendations

[23, 45, 67, 58, 54]

In [18]:
hero_objects = []
for hero_id in recommendations:
    for heroData in heroesData:
        if heroData["id"] == hero_id:
            hero_objects.append(heroData);
            
print hero_objects


[{u'localized_name': u'Kunkka', u'name': u'npc_dota_hero_kunkka', u'id': 23}, {u'localized_name': u'Pugna', u'name': u'npc_dota_hero_pugna', u'id': 45}, {u'localized_name': u'Spectre', u'name': u'npc_dota_hero_spectre', u'id': 67}, {u'localized_name': u'Enchantress', u'name': u'npc_dota_hero_enchantress', u'id': 58}, {u'localized_name': u'Lifestealer', u'name': u'npc_dota_hero_life_stealer', u'id': 54}]


In [ ]:
import os, logging, argparse
from dota2py import api
from pymongo import MongoClient
from time import sleep
from sys import exit
import env

In [29]:
os.environ['DOTABOT_API_KEY'] = "6E83EFA8D7815C7747AADD786AA043E8"
os.environ['DOTABOT_DB_SERVER'] = "127.0.0.1"
os.environ["DOTABOT_DB_NAME"] = 'dota2'

In [30]:
client = MongoClient(os.getenv('DOTABOT_DB_SERVER','localhost'), 27017)
db = client[os.getenv('DOTABOT_DB_NAME')]
match_collection = db.matches


In [31]:
match_collection

Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), u'dota2'), u'matches')

In [32]:
logging.basicConfig(filename='log.txt')
logger = logging.getLogger('dotabot')

In [37]:
def setup():
    """ Setup the API, etc."""
    logger.setLevel(logging.DEBUG)
    
    API_KEY = os.getenv("DOTABOT_API_KEY")
    if not API_KEY:
        raise NameError('Please set the DOTABOT_API_KEY environment variables.')
    api.set_api_key(API_KEY)

In [38]:
def is_valid_match(gmd_result):
    """
    Returns True if the given match details result should be considered,
    and False otherwise.
    """
    for player in gmd_result["players"]:
        if player["leaver_status"] is not 0:
            return False
    return True

In [39]:
def process_match_details(match_id):
    """
    Get the details of the given match_od, check if it's valid, and
    if it is, add it as a record in the database and spawn a thread to 
    download and parse the corresponding replay.
    """
    gmd = api.get_match_details(match_id)['result']
    
    if not is_valid_match(gmd):
        logger.debug('Not considering match %s.' % match_id)
        return
    
    match_collection.insert(gmd)

In [42]:
## Setup before start
setup()

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'dota2'), u'matches')

In [43]:
##Main
""" The main entry point of dotabot."""
start_match_id = None
while True:
    # Note: GetMatch History returns a list of mathces in descending order
    # going back in time.
    sleep(1.0)
    logger.debug('Doing GMH query for start_at_match_id=%s' % start_match_id)
    gmh = api.get_match_history(start_at_match_id=start_match_id,
                                    skill=3,
                                    game_mode=2,
                                    min_players=10)['result']
    error_code = gmh['status']
    matches = gmh['matches']
    
    if error_code is not 1:
        msg = 'GMH query at match_id %s had error code %s. Retrying.' % (start_match_id, error_code)
        logger.debug(msg)
        continue
    
    if len(matches) is 0:
        logger.debug('Finished processing all 500 most recent matches.')
        exit(0)
        
    for match in matches:
        match_id = match['match_id']
        
        if match_collection.find_one({'match_id': match_id}) != None:
            logger.debug('Encountered match %s already in database, exiting.' % match_id)
            exit(0)
        
        sleep(1.0)
        process_match_details(match_id)
    
    last_match_id = matches[-1]['match_id']
    logger.debug('Match_id of last match of GMH query: %s' % last_match_id)
    start_match_id = last_match_id -1

DEBUG:dotabot:Doing GMH query for start_at_match_id=None
DEBUG:dotabot:Not considering match 2207151204.
DEBUG:dotabot:Not considering match 2207147485.
DEBUG:dotabot:Not considering match 2207139419.
DEBUG:dotabot:Not considering match 2207139416.
DEBUG:dotabot:Not considering match 2207128559.
DEBUG:dotabot:Not considering match 2207127209.
DEBUG:dotabot:Not considering match 2207127134.
DEBUG:dotabot:Not considering match 2207126765.
DEBUG:dotabot:Not considering match 2207123309.
DEBUG:dotabot:Not considering match 2207121536.
DEBUG:dotabot:Not considering match 2207120953.
DEBUG:dotabot:Not considering match 2207120440.
DEBUG:dotabot:Not considering match 2207120368.
DEBUG:dotabot:Not considering match 2207115987.
DEBUG:dotabot:Not considering match 2207114836.
DEBUG:dotabot:Not considering match 2207114747.
DEBUG:dotabot:Not considering match 2207114601.
DEBUG:dotabot:Not considering match 2207114556.
DEBUG:dotabot:Not considering match 2207113949.
DEBUG:dotabot:Not considering m

KeyboardInterrupt: 